In [29]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from ast import literal_eval
import json

In [30]:
train = "../Datasets/spaced_catslu/train.csv"
vad = "../Datasets/spaced_catslu/development.csv"
test = "../Datasets/spaced_catslu/test.csv"

In [31]:
train = pd.read_csv(train, header=0)
vad = pd.read_csv(vad, header=0)
test = pd.read_csv(test, header=0)

In [32]:
train['transcript'][0]

'n a l ts a ɪ n ʁ i m iː ʐ ə ʁ w a lʲ ə v a ə'

In [33]:
def build_phone_vocab(df_all, temp, cnt_):
    length = len(train['transcript'])
    cnt = cnt_
    for i in range(length):
        if type(train['transcript'][i]) != str:
            continue
        for phone in train['transcript'][i]:
            if phone not in temp:
                temp[phone] = cnt
                cnt += 1
    return temp, cnt

In [9]:
def save_phone_idx(phone2idx, save_dir="../memory/cn"):
    with open(os.path.join(save_dir, "phone_idx.json"), 'w', encoding='utf-8') as f:
        json.dump(phone2idx, f)
    print("phone_idx.json saved in {}".format(os.path.abspath(save_dir)))

In [10]:
phone2idx, cnt = build_phone_vocab(train, dict(), 0)
phone2idx, cnt = build_phone_vocab(vad, phone2idx, cnt)
phone2idx, cnt = build_phone_vocab(test, phone2idx,cnt)

In [11]:
save_phone_idx(phone2idx)

phone_idx.json saved in /home/ubuntu/SLU-M-BERT/memory/cn


In [12]:
len(phone2idx)

78

### Save data as "train_x.npy" and "train_y.npy"

In [34]:
def load_phone_idx(file_path="../memory/cn/phone_idx.json"):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [35]:
def read_data(filename):
    return pd.read_csv(filename)

In [36]:
def get_xy(df, phone2idx):
    length = len(df)
    labels = df['label']
    labels_return = list()
    df = df['transcript']
    X_all = list()
    for i in range(length):
        if type(df[i]) != str:
            continue
        sample = df[i]
        x = [phone2idx[phone] for phone in sample]
        X_all.append(x)
        labels_return.append(labels[i])
    X_all = np.array(X_all, dtype=object)
    return X_all, labels_return

In [37]:
phone2idx = load_phone_idx("../memory/cn/phone_idx.json")

In [38]:
train_x, train_labels = get_xy(train, phone2idx)
vad_x, vad_labels = get_xy(vad, phone2idx)
test_x, test_labels = get_xy(test, phone2idx)

In [40]:
def save_numpy_data(filename, data):
    with open(filename, 'wb') as f:
        np.save(f, data)

In [43]:
save_numpy_data("../memory/cn/train_x.npy", train_x)
save_numpy_data("../memory/cn/train_y.npy", train_labels)
save_numpy_data("../memory/cn/dev_x.npy", vad_x)
save_numpy_data("../memory/cn/dev_y.npy", vad_labels)
save_numpy_data("../memory/cn/test_x.npy", test_x)
save_numpy_data("../memory/cn/test_y.npy", test_labels)